# 1.载入数据

## 1.1载入总的合并数据

In [ ]:

from signal import signal
import numpy as np
import pandas as pd
import glob
from preprocessSpectral import preprocessSpectral
from  matplotlib import pyplot as plt
import scipy.signal as signal
import sys
import zipfile
# 标记字典
dfs = []
labelDict = {'1': 1,'2':2,'3':3,'G11':4,'G12':5,'G13':6}

# 读入数据
xlsxfiles = glob.glob('hyperdata/224_531/*.xlsx')
sd = pd.DataFrame()
avgs = pd.DataFrame()
for file in xlsxfiles:
    filename = file.split('\\')[1]
    label = labelDict[filename.split('_')[0]]
    data = pd.read_excel(file,index_col = 0)
    data=data.transpose()
    

    data = data[~np.isnan(data).any(axis=1)]
    data = data[~np.isinf(data).any(axis=1)]
    
    '''选择部分行数据'''
    # data=data.iloc[20:50,:]

    '''原始的数据,没有预处理'''
    data_process = data.to_numpy()
    '''带预处理的数据'''
    # data_process = preprocessSpectral.MSC(data.to_numpy())

    # data_process = preprocessSpectral.SG(data_process,31,1)

    # data_process = preprocessSpectral.SG(data_process,17,1)
 




    # data_process = preprocessSpectral.D1(data_process)
    # data_process = signal.detrend(data_process)
    
    print(file)
    print(data.shape)
    # 保存图像
    # plt.plot(np.transpose(data_process)) 
    plt.savefig(sys.path[0]+ "/hyperdata/data_preprocess/" + filename.replace('xlsx', 'jpg'))
    plt.clf()
    # 加入数据
    data = pd.DataFrame(data_process)
    # 标准差
    sd[filename.split('_')[0]] = data.std(axis=0)
    # avgs[filename.split('.')[0]] = data.mean(axis=0)
    data.loc[:, 'label'] = label
    dfs.append(data) 
    pass


## 1.2数据做PCA运算

In [ ]:
datas = pd.concat(dfs,axis=0)
datas

In [ ]:
dataspca=datas.iloc[:,:-1]
dataspca

In [ ]:
labels = datas['label']

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=23)
X_pca = pca.fit_transform(dataspca)

# 打印每个主成分的贡献率
print(pca.explained_variance_ratio_)

In [ ]:
X_pca.shape

In [ ]:
datas=X_pca

In [ ]:
datas=pd.DataFrame(datas)
datas

In [ ]:
labels

In [ ]:
datas = datas.reset_index()
labels = labels.reset_index()

In [ ]:
datalabels = pd.concat([datas,labels],axis=1)
datalabels

In [ ]:
datalabels=datalabels.drop('index',axis=1)

In [ ]:
datalabels

## 1.3SPA算法

In [ ]:
datas = pd.concat(dfs,axis=0)
datas

In [ ]:
datasspa=datas.iloc[:,:-1]
datasspa

In [ ]:
labels = datas['label']

顺序1-SPA得到的原始结果

In [ ]:
datasspa=datasspa.iloc[:,[138,124,  18,  54,  79, 150, 106, 169, 178, 203, 164, 130, 201,
         6, 194, 206, 196, 200, 199, 205, 204]]

In [ ]:
datasspa

In [ ]:
datalabels = pd.concat([datasspa,labels],axis=1)
datalabels

顺序2-SPA得到的结果按顺序排列

In [ ]:
datass=datasspa.iloc[:,[  6, 18,  54,  79,106,124, 130,138,  150, 164,  169, 178, 194,196,199,   200,  201,203, 
         204, 205, 206]]


In [ ]:
datas = pd.concat([datass,labels],axis=1)

## 1.4数据的拼接--统一跳转这里

In [ ]:
datas = pd.concat(dfs,axis=0)
datas

In [ ]:
datas.to_excel("meizhong100.xlsx")

In [ ]:
datas= datas.drop(index=23)

## 1.5画图 

每一类高粱画光谱图

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# 假设你有一个名为df的DataFrame，包含光谱数据和标签

# 提取光谱数据和标签
spectra = datas.iloc[:, :-1]  # 光谱数据，去掉最后一列
labels = datas.iloc[:, -1]    # 标签，最后一列

# 获取不同类别的标签
unique_labels = labels.unique()
class_labels = ['Lu1','Lang19','Yi4','Jiu9','Ai','Kang']
# 设置绘图的颜色列表
colors = ['#1F77B4', '#FF7F0E', '#2CA02C', '#9467BD','#E377C2','yellow']

# 绘制不同类别的光谱
for label, color in zip(class_labels, colors):
    plt.plot([], [], color=color, label=label)  # 添加空线条，只用于图例

# 添加图例
plt.legend()
for label, color in zip(unique_labels, colors):
    # 获取相应类别的光谱数据
    label_spectra = spectra[labels == label]
    
    # 绘制光谱线条
    plt.plot(label_spectra.T, color=color, label=label)

# 添加图例


# 显示图像
plt.show()


每一类高粱平均后画光谱图

In [ ]:
sd2 = pd.DataFrame()
fig, ax = plt.subplots()
labels = ['Lu1','Lang19','Yi4','Jiu9','Ai','Kang']
plt.figure(figsize = (10,10))
for i in range(0,6):
    sd2[i+1]=datas[(datas['label']==i+1)].iloc[:,:-1].mean(axis=0)
    ax.plot(sd2[i+1], label=labels[i])
    # sd[filename.split('.')[0]] = data.std(axis=0)
ax.legend()
ax.set_xlabel('spectral')
ax.set_ylabel('refelction')
ax.set_title('spectral image')

In [ ]:
sd2.plot(title='mean')


画热力图

In [ ]:
sd2=sd2.transpose()

In [ ]:
from matplotlib.colors import ListedColormap
colors = ['blue', 'green', 'yellow', 'orange', 'red']
cmap = ListedColormap(colors)
heatmap = plt.imshow(sd2, cmap='viridis', aspect='auto')

# plt.colorbar()  # 添加颜色条
labels = ['Lu1','Lang19','Yi4','Jiu9','Ai','Kang']
plt.colorbar(heatmap, cmap=cmap)
categories =['Lu1','Lang19','Yi4','Jiu9','Ai','Kang']
plt.xticks(np.arange(len(categories)), categories)
# colorbar = plt.colorbar(heatmap)
# colorbar.set_label('New Label') 
plt.title('Hyperspectral Heatmap')
plt.ylabel('spectral')
plt.xlabel('Categories')

plt.show()


## 1.6组装数据

In [ ]:
# ovr分类使用
labels = datas['label']
datas = datas.drop(columns='label')
datalabels = pd.concat([datas,labels],axis=1)
datalabels

one-hot编码

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
# one-hot编码多分类使用,在这里改标签
labels = pd.get_dummies(datas['label'])
classmap = {'1': 1,'2':2,'3':3,'G11':4,'G12':5,'G13':6}#,'G3':4,'G5':5,'G6':6,'G7':7,'G8':8,'G9':9,'G10':10,'G11':11,'G12':12,'G13':13,'G14':14
labels = labels.rename(columns=classmap)
labels

## 1.7保存数据

In [ ]:
datas.to_excel("所有光谱.xlsx")

In [ ]:
sd1.to_excel("224维光谱6.xlsx")

In [ ]:
sd2.to_excel("预处理之后的平均光谱.xlsx")

In [ ]:
sd2.to_excel("224维光谱没平均.xlsx")

# 载入测试集数据，验证实验

## 导入验证集

In [ ]:
'''
Description: 
Version: 1.0
Author: tangliwen
Date: 2022-09-27 08:41:08
LastEditors: tangliwen
LastEditTime: 2022-10-20 14:03:27
'''
from signal import signal
import numpy as np
import pandas as pd
import glob
from preprocessSpectral import preprocessSpectral
from  matplotlib import pyplot as plt
import scipy.signal as signal
import sys
import zipfile
# 标记字典
dfs2 = []
labelDict = {'1': 1,'2':2,'3':3,'G11':4,'G12':5,'G13':6}
classes = ['宜糯红4号','泸州红1号','郎糯红19号']
# 读入数据
xlsxfiles = glob.glob('hyperdata/验证数据2/*.xlsx')
sd = pd.DataFrame()
avgs = pd.DataFrame()
for file in xlsxfiles:
    filename = file.split('\\')[1]
    label2 = labelDict[filename.split('_')[0]]
    data2 = pd.read_excel(file,index_col = 0)
    data2=data2.transpose()
    

    data2 = data2[~np.isnan(data2).any(axis=1)]
    data2 = data2[~np.isinf(data2).any(axis=1)]
    # print(data.shape)
    # 预处理,wave,MSC,
    # data_process=data


    # data_process = preprocessSpectral.SG(data.to_numpy(),11,1)
    # data_process = preprocessSpectral.MSC(data_process)
    # data_process = preprocessSpectral.MSC(data_process)
    # data_process = preprocessSpectral.D1(data.to_numpy())
    data_process2 = preprocessSpectral.MSC(data2.to_numpy())

    data_process2 = preprocessSpectral.SG(data_process2,11,1)

    data_process2 = preprocessSpectral.SG(data_process2,17,1)
 




    # data_process = preprocessSpectral.D1(data_process)
    # data_process = signal.detrend(data_process)
    
    print(file)
    print(data2.shape)
    # 保存图像
    # plt.plot(np.transpose(data_process)) 
    plt.savefig(sys.path[0]+ "/hyperdata/data_preprocess/" + filename.replace('xlsx', 'jpg'))
    plt.clf()
    # 加入数据
    data2 = pd.DataFrame(data_process2)
    # 标准差
    sd[filename.split('_')[0]] = data2.std(axis=0)
    # avgs[filename.split('.')[0]] = data.mean(axis=0)
    data2.loc[:, 'label'] = label
    dfs2.append(data2) 
    pass


In [ ]:
datas2 = pd.concat(dfs2,axis=0)
datas2

In [ ]:
# ovr分类使用
labels2 = datas2['label']
datas2 = datas2.drop(columns='label')
datalabels2 = pd.concat([datas2,labels2],axis=1)
datalabels2

# 2.划分训练集和测试集

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
train_data = []
train_label = []
test_data = []
test_label = []

# classes = ['宜糯红4号','泸州红1号','郎糯红19号']
for i in range(1,7):
    # traindata,testdata = train_test_split(datalabels[datalabels['label'] == i],test_size=0.2,random_state=0,shuffle=True)
    traindata=datalabels[datalabels['label'] == i]
    testdata=datalabels2[datalabels2['label'] == i]
    train_data.append(traindata.drop(columns='label'))
    train_label.append(traindata['label'])
    test_data.append(testdata.drop(columns='label'))
    test_label.append(testdata['label'])


train_pd = pd.DataFrame(columns=['datapath','label'])
test_pd = pd.DataFrame(columns=['datapath','label'])
for i in range(0,len(train_data)):
    train_pd = train_pd.append(pd.Series({'datapath':train_data[i],'label':train_label[i]}),ignore_index=True)
    pass
for i in range(0,len(test_data)):
    test_pd = test_pd.append(pd.Series({'datapath':test_data[i],'label':test_label[i]}),ignore_index=True)
    pass
train_pd = shuffle(train_pd)
test_pd = shuffle(test_pd)

train_data_all = pd.concat(train_data,axis=0)
test_data_all = pd.concat(test_data,axis=0)
train_label_all = pd.concat(train_label,axis=0)
test_label_all = pd.concat(test_label,axis=0)

## 2.1 手动指定

In [ ]:
samples = 200
startindexs = []
for i in range(int(datas.shape[0]/samples)):
    startindexs.append(i * samples)
startindexs

# 生成测试的索引
test_data_index = []
train_data_index = []
for i in range(len(startindexs)):   
    if((i-2)%4!=0):
        train_data_index.append(i)       
    else:
        test_data_index.append(i)
pass

# 生成训练集与测试集数据
train_data = []
train_label = []
test_data = []
test_label = []
for i in train_data_index:
    index = startindexs[i]
    train_data.append(pd.DataFrame(datas.iloc[index:index+samples,:]))
    train_label.append(pd.DataFrame(labels.iloc[index:index+samples,:]))
for i in test_data_index:
    index = startindexs[i]
    test_data.append(pd.DataFrame(datas.iloc[index:index+samples,:]))
    test_label.append(pd.DataFrame(labels.iloc[index:index+samples,:]))
train_data_all = pd.concat(train_data,axis=0)
test_data_all = pd.concat(test_data,axis=0)
train_label_all = pd.concat(train_label,axis=0)
test_label_all = pd.concat(test_label,axis=0)

In [ ]:
test_data_all = datas
test_label_all = labels

## 2.2 自动分配

### 2.2.1 带PCA降维

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
import joblib
train_data = []
train_label = []
test_data = []
test_label = []
pcas = []
classes = ['1': 0,'2':1,'3':2]
for i in range(3):
    pca = PCA(n_components=20)

    traindata,testdata = train_test_split(datalabels[datalabels[classmap[i]] == 1],test_size=0.2,random_state=0,shuffle=True)
    train_data.append(pd.DataFrame(pca.fit_transform(traindata.drop(columns=classes))))
    train_label.append(traindata[classes])
    test_data.append(pd.DataFrame(pca.transform(testdata.drop(columns=classes))))
    test_label.append(testdata[classes])
    
    joblib.dump(pca,'hyperdata/outmodels/PCA/'+classes[i]+'.pkl')
    pcas.append(pca)
    # PCA降维
    print(pca.explained_variance_ratio_.sum())
    # 

train_data_all = pd.concat(train_data,axis=0)
test_data_all = pd.concat(test_data,axis=0)
train_label_all = pd.concat(train_label,axis=0)
test_label_all = pd.concat(test_label,axis=0)
    

### 2.2.2 不带降维

#### 2.2.2.1 one-hot类型的测试训练集分割

In [ ]:
from sklearn.model_selection import train_test_split

train_data = []
train_label = []
test_data = []
test_label = []

classes = ['021','bainong207','fanmai8hao','quanmai725','weilong169']
for i in range(5):
    traindata,testdata = train_test_split(datalabels[datalabels[classmap[i]] == 1],test_size=0.2,random_state=0,shuffle=False)
    train_data.append(traindata.drop(columns=classes))
    train_label.append(traindata[classes])
    test_data.append(testdata.drop(columns=classes))
    test_label.append(testdata[classes])
    
train_data_all = pd.concat(train_data,axis=0)
test_data_all = pd.concat(test_data,axis=0)
train_label_all = pd.concat(train_label,axis=0)
test_label_all = pd.concat(test_label,axis=0)
    

In [ ]:
datalabels

#### 2.2.2.2 OVR型测试训练集分割

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
train_data = []
train_label = []
test_data = []
test_label = []

# classes = ['宜糯红4号','泸州红1号','郎糯红19号']
for i in range(1,7):
    traindata,testdata = train_test_split(datalabels[datalabels['label'] == i],test_size=0.2,random_state=0,shuffle=True)
    train_data.append(traindata.drop(columns='label'))
    train_label.append(traindata['label'])
    test_data.append(testdata.drop(columns='label'))
    test_label.append(testdata['label'])


train_pd = pd.DataFrame(columns=['datapath','label'])
test_pd = pd.DataFrame(columns=['datapath','label'])
for i in range(0,len(train_data)):
    train_pd = train_pd.append(pd.Series({'datapath':train_data[i],'label':train_label[i]}),ignore_index=True)
    pass
for i in range(0,len(test_data)):
    test_pd = test_pd.append(pd.Series({'datapath':test_data[i],'label':test_label[i]}),ignore_index=True)
    pass
train_pd = shuffle(train_pd)
test_pd = shuffle(test_pd)

train_data_all = pd.concat(train_data,axis=0)
test_data_all = pd.concat(test_data,axis=0)
train_label_all = pd.concat(train_label,axis=0)
test_label_all = pd.concat(test_label,axis=0)

In [ ]:
from sklearn.model_selection import train_test_split

train_data = []
train_label = []
test_data = []
test_label = []

# classes = ['021','bainong207','fanmai8hao','quanmai725','weilong169']
for i in range(1,4):
    traindata,testdata = train_test_split(datalabels[datalabels['label'] == i],test_size=0.2,random_state=0,shuffle=True)
    train_data.append(traindata.drop(columns='label'))
    train_label.append(traindata['label'])
    test_data.append(testdata.drop(columns='label'))
    test_label.append(testdata['label'])
    
train_data_all = pd.concat(train_data,axis=0)
test_data_all = pd.concat(test_data,axis=0)
train_label_all = pd.concat(train_label,axis=0)
test_label_all = pd.concat(test_label,axis=0)

In [ ]:
train_label=np.array(train_label)
test_label=np.array(test_label)

In [ ]:
for i in range(0,len(train_data)):
    train_pd = train_pd.append(pd.Series({'datapath':train_data[i],'label':train_label[i]}),ignore_index=True)
    pass
for i in range(0,len(test_data)):
    test_pd = test_pd.append(pd.Series({'datapath':test_data[i],'label':test_label[i]}),ignore_index=True)
    pass

train_pd = shuffle(train_pd)
test_pd = shuffle(test_pd)
train_data = np.array(train_pd['datapath'])
train_label = np.array(pd.get_dummies(train_pd['label']))
test_data = np.array(test_pd['datapath'])
test_label = np.array(pd.get_dummies(test_pd['label']))
# 训练模型
print(train_data.shape)
print(train_label.shape)
print(test_data.shape)
print(test_label.shape)


In [ ]:
test_label_all.shape

### 2.2.3 带波长选择的集合划分

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
import joblib
train_data = []
train_label = []
test_data = []
test_label = []
# 波长选择
waves = []
# 手动根据STD指定
waves = np.arange(0,25,1)
waves = np.append(waves,np.arange(90, 110, 1),0)
waves = np.append(waves,np.arange(135, 165, 1),0)
waves = np.append(waves,np.arange(180, 190, 1),0)
classes = ['021','bainong207','fanmai8hao','quanmai725','weilong169']
for i in range(5):
    traindata,testdata = train_test_split(datalabels[datalabels[classmap[i]] == 1],test_size=0.2,random_state=0,shuffle=True)
    train_data.append(pd.DataFrame(traindata.drop(columns=classes)[waves]))
    train_label.append(traindata[classes])
    test_data.append(pd.DataFrame(testdata.drop(columns=classes)[waves]))
    test_label.append(testdata[classes])
    pass
train_data_all = pd.concat(train_data,axis=0)
test_data_all = pd.concat(test_data,axis=0)
train_label_all = pd.concat(train_label,axis=0)
test_label_all = pd.concat(test_label,axis=0)

### 2.2.4 带波长选择的OVR

In [ ]:
sd.plot()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
import joblib
train_data = []
train_label = []
test_data = []
test_label = []
# 波长选择
waves = []
# 手动根据STD指定
waves = np.arange(0,5,1)
waves = np.append(waves,np.arange(95, 105, 1),0)
waves = np.append(waves,np.arange(145, 155, 1),0)
waves = np.append(waves,np.arange(195, 200, 1),0)
classes = ['021','bainong207','fanmai8hao','quanmai725','weilong169']
for i in range(5):
    traindata,testdata = train_test_split(datalabels[datalabels['label'] == i],test_size=0.2,random_state=0,shuffle=True)
    train_data.append(pd.DataFrame(traindata.drop(columns='label')[waves]))
    train_label.append(traindata['label'])
    test_data.append(pd.DataFrame(testdata.drop(columns='label')[waves]))
    test_label.append(testdata['label'])
    pass
train_data_all = pd.concat(train_data,axis=0)
test_data_all = pd.concat(test_data,axis=0)
train_label_all = pd.concat(train_label,axis=0)
test_label_all = pd.concat(test_label,axis=0)

# 3. 特征工程

In [ ]:
plt.plot(newX[200:400,1])

In [ ]:
import seaborn as sns
plt.figure(figsize=(50,50))
corr = datalabels.corr()
ax = sns.heatmap(corr, xticklabels=corr.columns, yticklabels=corr.columns, cmap="RdYlGn",annot=False)
plt.title("Correlation between variables")

# 4.建模

## 4.1 用之前的模型建模

In [ ]:
import joblib
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix,accuracy_score,precision_score
from sklearn.metrics import classification_report
# clf = DecisionTreeClassifier(random_state=0)
# rfc = RandomForestClassifier(random_state=0)
# rfc = rfc.fit(train_data_all,train_label_all)
# 载入
rfc = joblib.load('hyperdata/models/clf.pkl')
y_pred = rfc.predict(test_data_all)
print(classification_report(test_label_all, y_pred))
print(accuracy_score(test_label_all, y_pred))
rfc.score(test_data_all,test_label_all)

## 4.2 建立模型

In [ ]:
from OpenSA.WaveSelect.WaveSelect import SpctrumFeatureSelcet
from OpenSA.WaveSelect.Cars import CARS_Cloud
wavefeaturelist = []

for i in range(5):
    idx = CARS_Cloud(datas.to_numpy(),labels.to_numpy()[:,i])
    wavefeaturelist.append(idx)
wavefeaturelist 

In [ ]:
s = pd.DataFrame(wavefeaturelist)
s
s.to_excel('hyperdata/models/carsfeature.xlsx')

In [ ]:
from OpenSA.Preprocessing.Preprocessing import Preprocessing
from OpenSA.WaveSelect.WaveSelect import SpctrumFeatureSelcet
from OpenSA.Simcalculation.SimCa import Simcalculation
from OpenSA.Clustering.Cluster import Cluster
from OpenSA.Regression.Rgs import QuantitativeAnalysis
from OpenSA.Classification.Cls import QualitativeAnalysis
from OpenSA.DataLoad.DataLoad import SetSplit, LoadNirtest
import warnings
warnings.filterwarnings('ignore') #  忽略弹出的warnings信息
def SpectralQualitativeAnalysis(data, label, ProcessMethods, FslecetedMethods, SetSplitMethods):
    ProcesedData = Preprocessing(ProcessMethods, data)
    FeatrueData, labels = SpctrumFeatureSelcet(FslecetedMethods, ProcesedData, label)
    X_train, X_test, y_train, y_test = SetSplit(SetSplitMethods, FeatrueData, labels, test_size=0.2,randomseed=0)    
    return X_train, X_test, y_train, y_test

waveselects = ['None','CARS','LARS','UVE','SPA','PCA']
models = ['LDA','QDA','XGBOOST','ADABOOST','KNN','GBDT','ANN']
for ws in waveselects:  
    results = pd.DataFrame(columns=models)
    for i in range(5): 
        X_train, X_test, y_train, y_test = SpectralQualitativeAnalysis(datas.to_numpy(),labels.to_numpy()[:,i],'None',ws,'random')
        modelresults = pd.Series()   
        for modelname in models:  
            accuracy = QualitativeAnalysis(modelname, X_train, X_test, y_train, y_test)
            modelresults[modelname] = accuracy
            print(modelname + '-' + str(i) + "-" + ws)
        pass
        results = results.append(modelresults,ignore_index=True) 
    pass
    results.to_excel(ws + '.xlsx')
pass
results


### 4.2.1 单独测试

In [ ]:
from OpenSA.Preprocessing.Preprocessing import Preprocessing
from OpenSA.WaveSelect.WaveSelect import SpctrumFeatureSelcet
from OpenSA.Simcalculation.SimCa import Simcalculation
from OpenSA.Clustering.Cluster import Cluster
from OpenSA.Regression.Rgs import QuantitativeAnalysis
from OpenSA.Classification.Cls import QualitativeAnalysis
from OpenSA.DataLoad.DataLoad import SetSplit, LoadNirtest
from sklearn.decomposition import PCA
import joblib
import warnings
warnings.filterwarnings('ignore') #  忽略弹出的warnings信息

# waveselects = ['None','CARS','LARS','UVE','SPA','PCA']
models = ['PLS_DA','LDA','QDA','XGBOOST','ADABOOST','KNN','GBDT','RF']
results = pd.DataFrame(columns=models)
for i in range(5): 
    modelresults = pd.Series()   
    for modelname in models:  
        accuracy,tmodel = QualitativeAnalysis(modelname, train_data_all, test_data_all, train_label_all[classmap[i]], test_label_all[classmap[i]]) 
        modelresults[modelname] = accuracy
        print(modelname + '-' + str(i))
        joblib.dump(tmodel, 'hyperdata/outmodels/'+modelname+'-'+str(classmap[i])+'-'+'20221026'+'.pkl')
    pass
    results = results.append(modelresults,ignore_index=True) 
pass
results.to_excel('NONE.xlsx')
results


In [ ]:
QualitativeAnalysis('RF', train_data_all, test_data_all, train_label_all, test_label_all )

### 4.2.2 OVR多分类器测试

In [ ]:
import joblib
from sklearn.decomposition import PCA
from sklearn.multiclass import OneVsRestClassifier

from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.svm import LinearSVR
from sklearn.svm import NuSVC
from sklearn.svm import NuSVR
from sklearn.svm import SVC
from sklearn.svm import SVR
from sklearn.gaussian_process import GaussianProcessClassifier

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.linear_model import SGDClassifier
import warnings
warnings.filterwarnings('ignore') #  忽略弹出的warnings信息

# clf = OneVsOneClassifier(LinearSVC(),n_jobs=-1)
models = [
    # QDA(),
    # LDA(),
    # RandomForestClassifier(),
    # DecisionTreeClassifier(),
    # ExtraTreeClassifier(),
    GradientBoostingClassifier(),  
    # ExtraTreesClassifier(),
    # # GaussianProcessClassifier(multi_class = 'one_vs_rest'),
    # LogisticRegression(multi_class='ovr'),
    # LogisticRegressionCV(multi_class='ovr'),
    
]
'''查看在测试集的表现'''
# result = pd.DataFrame(columns=['model','acc','recall','f1_score'])
# for model in models:
#     clf = OneVsRestClassifier(model,n_jobs=-1)
#     clf.fit(train_data_all,train_label_all)
#     y_pred = clf.predict(test_data_all)
#     print(model.__class__)
#     joblib.dump(clf, 'hyperdata/outmodels/OVR-models/'+model.__class__.__name__+'.pkl')
#     with open('result.txt', 'a') as f:  # 设置文件对象
#         print(model.__class__,file=f)    
#         print(classification_report(test_label_all,y_pred),file=f)
        
#     result = result.append(pd.Series({
#         'model' : model.__class__,
#         'acc':accuracy_score(test_label_all,y_pred),
#         'recall':recall_score(test_label_all, y_pred,average='macro'),
#         'f1_score':f1_score(test_label_all, y_pred,average='macro'),
#         }),ignore_index=True)
    
'''查看在训练集的表现'''
result = pd.DataFrame(columns=['model','acc','recall','f1_score'])
for model in models:
    clf = OneVsRestClassifier(model,n_jobs=-1)
    clf.fit(train_data_all,train_label_all)
    y_pred = clf.predict(train_data_all)
    print(model.__class__)
    joblib.dump(clf, 'hyperdata/outmodels/OVR-models/'+model.__class__.__name__+'.pkl')
    with open('result.txt', 'a') as f:  # 设置文件对象
        print(model.__class__,file=f)    
        print(classification_report(train_label_all,y_pred),file=f)
        
    result = result.append(pd.Series({
        'model' : model.__class__,
        'acc':accuracy_score(train_label_all,y_pred),
        'recall':recall_score(train_label_all, y_pred,average='macro'),
        'f1_score':f1_score(train_label_all, y_pred,average='macro'),
        }),ignore_index=True)
result

查看模型参数量

In [ ]:
params_num = len(model.get_params())

# 输出参数数量
print("Model的参数量为：", params_num)

测试集结果画图

In [ ]:
import seaborn as sns
data = confusion_matrix(test_label_all, y_pred)
names=['Lu1','Lang19','Yi4','Jiu9','Ai','Kang']
df_cm = pd.DataFrame(data, columns=np.unique(names), index = np.unique(names))
df_cm.index.name = 'Actual'
df_cm.columns.name = 'Predicted'

plt.figure(figsize = (10,8))
sns.set(font_scale=1.4)#for label size
sns.heatmap(df_cm, cmap="Reds", annot=True,annot_kws={"size": 16}, fmt='d')

plt.savefig('cmap.png', dpi=300)

训练集结果画图

In [ ]:
print(classification_report(test_label_all, y_pred))
accuracy_score(test_label_all,y_pred)

In [ ]:
print(classification_report(train_label_all, y_pred))
accuracy_score(train_label_all,y_pred)

In [ ]:
import seaborn as sns
data = confusion_matrix(train_label_all, y_pred)
names=['Lu1','Lang19','Yi4','Jiu9','Ai','Kang']
df_cm = pd.DataFrame(data, columns=np.unique(names), index = np.unique(names))
df_cm.index.name = 'Actual'
df_cm.columns.name = 'Predicted'

plt.figure(figsize = (10,8))
sns.set(font_scale=1.4)#for label size
sns.heatmap(df_cm, cmap="Reds", annot=True,annot_kws={"size": 16}, fmt='d')

plt.savefig('cmap.png', dpi=300)

### 4.2.3 OVO多分类器

In [ ]:
import joblib
from sklearn.decomposition import PCA
from sklearn.multiclass import OneVsOneClassifier

from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.svm import LinearSVR
from sklearn.svm import NuSVC
from sklearn.svm import NuSVR
from sklearn.svm import SVC
from sklearn.svm import SVR
import warnings
warnings.filterwarnings('ignore') #  忽略弹出的warnings信息
result = pd.DataFrame(columns=['model','acc'])
# clf = OneVsOneClassifier(LinearSVC(),n_jobs=-1)
models = [QDA,LDA,RandomForestClassifier,LinearSVC,LinearSVR,NuSVC,NuSVR,SVC,SVR,DecisionTreeClassifier,ExtraTreeClassifier,
AdaBoostClassifier,GradientBoostingClassifier]
for model in models:
    clf = OneVsOneClassifier(model(),n_jobs=-1)
    clf.fit(train_data_all,train_label_all)
    y_pred = clf.predict(test_data_all)
    result = result.append(pd.Series({'model' : model.__name__,'acc':accuracy_score(test_label_all,y_pred)}),ignore_index=True)

# 
# rfc = RandomForestClassifier(random_state=123)
# rfc = rfc.fit(train_data_all,train_label_all)
# y_pred = rfc.predict(test_data_all)
result

In [ ]:
print(classification_report(test_label_all, y_pred))


## 4.3 保存模型

In [ ]:
# 保存模型
import joblib
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA

for k,v in labelDict.items():
    rfc = QDA()
    # CARS特征
    carsfeatures = pd.read_excel('hyperdata/models/carsfeature.xlsx')
    cf = carsfeatures.iloc[v,:].tolist()
    new_list=[]
    for elem in cf:
        if not np.isnan(elem):
            new_list.append(elem)
    rfc = rfc.fit(datas.iloc[:,new_list],labels.iloc[:,v])
    joblib.dump(rfc, 'hyperdata/models/QDA4000-v1.0-'+str(k)+'.pkl')

## 4.3 随机分组交叉验证

In [ ]:
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier as RF                       # 随机森林
def kFold_cv(X, y, classifier, **kwargs):
    """
    :param X: 特征
    :param y: 目标变量
    :param classifier: 分类器
    :param **kwargs: 参数
    :return: 预测结果
    """
    kf = KFold(n_splits=5, shuffle=True) 
    y_pred = np.zeros(shape=(len(y),5))         # 初始化y_pred数组
    
    for train_index, test_index in kf.split(X):  
        X_train = X[train_index]    
        X_test = X[test_index]
        y_train = y[train_index]      # 划分数据集
        clf = classifier(**kwargs)    
        clf.fit(X_train, y_train)     # 模型训练
        y_pred[test_index,:] = clf.predict(X_test)  # 模型预测
    
    return y_pred 

y_PRED = kFold_cv(datas.to_numpy(), labels.to_numpy(), RF)

# 4.评价模型

In [ ]:
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
def metrics_data(y,ypred):

    accuracy = accuracy_score(y, ypred)
    precision = precision_score(y, ypred,average='micro')
    recall = recall_score(y, ypred,average='micro')
    f1 = f1_score(y, ypred,average='micro')
    print('accuracy:{}'.format(accuracy))
    print('precision:{}'.format(precision))
    print('recall:{}'.format(recall))
    print('f1-score:{}'.format(f1))
    metrics = pd.DataFrame(columns=['accuracy','precision','recall','f1'])
    metrics = metrics.append(pd.Series({"accuracy":accuracy,"precision":precision,'recall':recall,'f1':f1}),ignore_index=True)
    print(classification_report(y, ypred))
    return metrics
metrics_data(test_label_all,ypred=y_pred)


# 4.保存模型

In [ ]:
import joblib

#保存Model(注:save文件夹要预先建立，否则会报错)
joblib.dump(rfc, 'hyperdata/models/clf.pkl')

#读取Model
clf3 = joblib.load('hyperdata/models/clf.pkl')

# 5.测试

In [ ]:
import joblib
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix,accuracy_score,precision_score
from sklearn.metrics import classification_report
# clf = DecisionTreeClassifier(random_state=0)
# rfc = RandomForestClassifier(random_state=0)
# rfc = rfc.fit(train_data_all,train_label_all)
# 载入
models = ['PLS_DA','LDA','QDA','XGBOOST','ADABOOST','KNN','GBDT','RF']
def loadMyModels(type='all',modelname='XGBOOST'):
    loadmodels = []
    pcamodel = []
    if type == 'all':
        for k,v in labelDict.items():
            loadmodels.append(joblib.load('hyperdata/outmodels/'+modelname+'-'+k+'-20221026.pkl'))
            pcamodel.append(joblib.load('hyperdata/outmodels/PCA/'+k+'.pkl'))
        pass
    else:
        loadmodels.append(joblib.load('hyperdata/outmodels/'+modelname+'-'+type+'-20221026.pkl'))
        pcamodel.append(joblib.load('hyperdata/outmodels/PCA/'+type+'.pkl'))

    return loadmodels,pcamodel

def useMyModels(loadmodels,x_test,y_test):
    loadModelAccuracys = []
    predResuls = pd.DataFrame()
    # CARS特征
    carsfeatures = pd.read_excel('hyperdata/models/carsfeature.xlsx')
    count = 0
    for m in loadmodels:
        cf = carsfeatures.iloc[count,:].tolist()
        new_list=[]
        for elem in cf:
            if not np.isnan(elem):
                new_list.append(elem)
        
        re = m.predict(x_test.iloc[:,new_list])
        predResuls[str(count)] = re
        count = count+1
    return predResuls

def useMyModels2(loadmodels,x_test,y_test,name):
    predResuls = pd.DataFrame()
    probResuls = pd.DataFrame()
    loadModelAccuracys = []
    count = 0
    for m in loadmodels:
        re = m.predict(x_test)
        # p = m.predict_proba(x_test)
        if name == 'PLS_DA':
            re = np.array([np.argmax(i) for i in re])
        # probResuls[classmap[count]] = p
        count = count + 1
    return re,probResuls

def useMyModels3(loadmodels,x_test,y_test,pcamodels,type,name):
    predResuls = pd.DataFrame()
    probResuls = pd.DataFrame()
    loadModelAccuracys = []
    for m in loadmodels:
        newx = pcamodels[0].transform(x_test)
        
        re = m.predict(newx)
        if name == 'PLS_DA':
             re = np.array([np.argmax(i) for i in re])
    return re,probResuls
# 
results = pd.DataFrame(columns=models)
for l in classes:
    modelresults = pd.Series() 
    for m in models:
        loadmodels,pcamodel = loadMyModels(l,m)
        pred,prob = useMyModels2(loadmodels=loadmodels,x_test=datas,y_test=labels,name=m)
        # PCA分类用
        # pred,prob = useMyModels3(loadmodels=loadmodels,x_test=datas,y_test=labels,pcamodels=pcamodel,type=l,name=m)
        modelresults[m] = precision_score(pred, labels[l])
    pass
    results = results.append(modelresults,ignore_index=True) 
pass
results.to_excel('NONE-预测集表现.xlsx')
results

## 5.1 OVR模型测试

In [ ]:
import joblib
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix,accuracy_score,precision_score
from sklearn.metrics import classification_report

from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.svm import LinearSVR
from sklearn.svm import NuSVC
from sklearn.svm import NuSVR
from sklearn.svm import SVC
from sklearn.svm import SVR
from sklearn.gaussian_process import GaussianProcessClassifier

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.linear_model import SGDClassifier
import warnings
warnings.filterwarnings('ignore') #  忽略弹出的warnings信息


def loadMyModels(modelname='XGBOOST'):
    return  joblib.load('hyperdata/outmodels/OVR-models/'+modelname+'.pkl')

def useMyModels(loadmodels,x_test,y_test):
    loadModelAccuracys = []
    re = loadmodels.predict(x_test.iloc[:,waves])
    # re = loadmodels.predict(x_test.iloc[:,:])
    return re

models = [
    # QDA(),
    LDA(),
    RandomForestClassifier(),
    SVC(),
    DecisionTreeClassifier(),
    ExtraTreeClassifier(),
    AdaBoostClassifier(),
    GradientBoostingClassifier(),
    # GaussianProcessClassifier(multi_class = 'one_vs_rest'),
    # LogisticRegression(multi_class='ovr'),
    # LogisticRegressionCV(multi_class='ovr'),
    ExtraTreesClassifier()
]

modelclsnames = []
for mod in models:
    modelclsnames.append(mod.__class__.__name__)

results = pd.DataFrame(columns=modelclsnames)
modelresults = pd.Series() 
for m in models:
    loadmodel = loadMyModels(m.__class__.__name__)
    pred= useMyModels(loadmodels=loadmodel,x_test=datas,y_test=labels)
    modelresults[m.__class__.__name__] = accuracy_score(pred, labels)
    with open('result_prd.txt', 'a') as f:  # 设置文件对象
        print(model.__class__,file=f)    
        print(classification_report(labels,pred),file=f)
pass
results = results.append(modelresults,ignore_index=True) 

results.to_excel('NONE-预测集表现.xlsx')
results

In [ ]:
loadmodelPred

# 6.深度学习分类

In [ ]:
import sklea